In [1]:
import torch
from preprocessing import Preprocessor
from training import Trainer, load_partial_model, load_full_model
from metrics import Metrics
import torch.nn as nn
import torch.optim as optim
#from models.triple_arch import N_Parallel_Models
from models.vgg_TL import GoogleNet
#from models.autoencoders import Simple_AE, CNN_VAE
from configuration import Hyperparameters as HP
import json
import math
import tabulate

years = [str(y) for y in range(2006, 2015)]

classes_30_cf = ["Asterionellopsis", "bad", "Chaetoceros", "Ciliate_mix", "Corethron", "Cylindrotheca", "Dictyocha","dino30", "detritus", 
    "Chaetoceros_flagellate", "Dinobryon", "Ditylum", "Eucampia", "flagellate_sp3", "Guinardia_delicatula", "Guinardia_flaccida", "Guinardia_striata",
    "Heterocapsa_triquetra", "Laboea_strobila", "Leptocylindrus","pennate", "Phaeocystis", "Pleurosigma", "Prorocentrum", "Pseudonitzschia", 
    "Skeletonema", "Thalassionema", "Thalassiosira", "Thalassiosira_dirty", "Tintinnid"]


In [2]:
pp = Preprocessor(years, include_classes=classes_30_cf, strategy = HP.pp_strategy, train_eg_per_class=HP.number_of_images_per_class, maxN = HP.maxN, 
    minimum =  HP.minimum, transformations = HP.transformations)


777655
29149


In [3]:
headers = ['classes', 'og_pp onehot index', 'conf_pp onehot index']
data = []
for c in classes_30_cf:
    data.append([c, pp.label_to_onehotInd(c), 0])

In [4]:
data

[['Asterionellopsis', 0, 0],
 ['bad', 25, 0],
 ['Chaetoceros', 1, 0],
 ['Ciliate_mix', 3, 0],
 ['Corethron', 4, 0],
 ['Cylindrotheca', 5, 0],
 ['Dictyocha', 6, 0],
 ['dino30', 27, 0],
 ['detritus', 26, 0],
 ['Chaetoceros_flagellate', 2, 0],
 ['Dinobryon', 7, 0],
 ['Ditylum', 8, 0],
 ['Eucampia', 9, 0],
 ['flagellate_sp3', 28, 0],
 ['Guinardia_delicatula', 10, 0],
 ['Guinardia_flaccida', 11, 0],
 ['Guinardia_striata', 12, 0],
 ['Heterocapsa_triquetra', 13, 0],
 ['Laboea_strobila', 14, 0],
 ['Leptocylindrus', 15, 0],
 ['pennate', 29, 0],
 ['Phaeocystis', 16, 0],
 ['Pleurosigma', 17, 0],
 ['Prorocentrum', 18, 0],
 ['Pseudonitzschia', 19, 0],
 ['Skeletonema', 20, 0],
 ['Thalassionema', 21, 0],
 ['Thalassiosira', 22, 0],
 ['Thalassiosira_dirty', 23, 0],
 ['Tintinnid', 24, 0]]

In [ ]:
t